In [10]:
import oracledb
import folium
import geojson

from shapely.wkt import loads

In [11]:
cs = oracledb.makedsn("dbmanage.lab.ii.agh.edu.pl", 1521, sid="DBMANAGE")

un = "student"
pw = "stu638dent"

connection = oracledb.connect(user=un, password=pw, dsn=cs)
connection.cursor().execute("ALTER SESSION SET CURRENT_SCHEMA = US_SPAT")

def output_type_handler(cursor, name, default_type, size, precision, scale):
    if default_type == oracledb.CLOB:
        return cursor.var(oracledb.LONG_STRING, arraysize = cursor.arraysize)
    
connection.outputtypehandler = output_type_handler

In [23]:
# Stan z największą ilością rzek

query = """
SELECT s.state, COUNT(*) as river_count
FROM us_states s, us_rivers r
WHERE SDO_ANYINTERACT(s.geom, r.geom) = 'TRUE'
GROUP BY s.state
ORDER BY river_count DESC
FETCH FIRST 1 ROWS ONLY
"""

results = connection.cursor().execute(query).fetchall()
print(results)

[('Alaska', 7)]


In [27]:
m = folium.Map()

query_1 = """
SELECT sdo_util.to_wktgeometry(s.geom)
FROM us_states s
where s.state = 'Alaska'
"""
results_1 = loads(connection.cursor().execute(query_1).fetchall())
st_1 = {'color': 'green', 'fillColor': 'green'}
l_1 = [geojson.Feature(geometry=row[0], propertries={}) for row in results_1]
f_1 = geojson.FeatureCollection(l_1)
folium.GeoJson(f_1, style_function=lambda x: st_1).add_to(m)

query_2 = """
SELECT sdo_util.to_wktgeometry(r.geom)
FROM us_states s, us_rivers r
WHERE SDO_ANYINTERACT(s.geom, r.geom) = 'TRUE' 
    and s.state = 'Alaska'
"""
results_2 = loads(connection.cursor().execute(query_2).fetchall())
st_2 = {'color': 'blue'}
l_2 = [geojson.Feature(geometry=row[0], propertries={}) for row in results_2]
f_2 = geojson.FeatureCollection(l_2)
folium.GeoJson(f_2, style_function=lambda x: st_2).add_to(m)              

m

In [30]:
# 5 największych miast USA i droga najbliżej każdego z nich

m = folium.Map()

query = """
SELECT sdo_util.to_wktgeometry(c.LOCATION),
       (select sdo_util.to_wktgeometry(GEOM)
        from US_INTERSTATES i
        order by SDO_GEOM.SDO_DISTANCE ( c.location, i.GEOM, 0.5)
        fetch first 1 row only) as GEOM
FROM US_CITIES c
order by c.POP90 desc
fetch first 5 rows only
"""

results = loads(connection.cursor().execute(query).fetchall())

l_1 = [geojson.Feature(geometry=row[0], propertries={}) for row in results]
f_1 = geojson.FeatureCollection(l_1)
folium.GeoJson(f_1, style_function=lambda x: st_1).add_to(m)

st_2 = {'color': 'grey'}
l_2 = [geojson.Feature(geometry=row[1], propertries={}) for row in results]
f_2 = geojson.FeatureCollection(l_2)
folium.GeoJson(f_2, style_function=lambda x: st_2).add_to(m)

m

In [32]:
# 5 stanów z największą łączną długością dróg

query = """
SELECT state,
       total_interstate_length
FROM (SELECT s.state,
             SUM(SDO_GEOM.SDO_LENGTH(SDO_GEOM.SDO_INTERSECTION(i.geom, s.geom, 0.005),
                                     0.005)) AS total_interstate_length
      FROM us_states s,
           us_interstates i
      WHERE SDO_RELATE(s.geom, i.geom, 'mask=ANYINTERACT') = 'TRUE'
      GROUP BY s.state)
ORDER BY total_interstate_length DESC
    FETCH FIRST 5 ROWS ONLY
"""

results = connection.cursor().execute(query).fetchall()
for res in results:
    print(res)

('Texas', 4994909.406738114)
('California', 3860816.064436342)
('Illinois', 3384822.0523314867)
('Pennsylvania', 2516674.1618397087)
('Ohio', 2475531.4114238084)
